Start with:
Data Check
Data cleansing
Verification that everything is proper

Analysis to do:
Country Analysis - Which country uses the most. BY $ amount and quantity
Product  Analytics  - Top products per country  
Price Analytics - By Month/Country
What sells the most, What is returned the most
Monthly ratio of successful sold to cancelled
Distribution between cancelled and successful invoice
Analysis of invoice by hourly distribution
Who are our top customers? By unit price, by quantity etc. Understand their needs and if data is skewed because of them eg. Big companies making big orders on a given product. (Might not mean it is favored by everyone). Basically check uniqueness in customer for given product.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import random
import os
import glob

%matplotlib inline
# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [2]:
ds = pd.read_excel("data_science_analytics_2018_data.xlsx")
#Save Orignal Data

In [3]:
ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
1,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
2,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
3,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
4,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397.0,United Kingdom


In [5]:
ds.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [21]:
ds.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
ds.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [15]:
ds.dropna(axis =0, subset=['CustomerID'],inplace=True)

In [16]:
ds.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

SyntaxError: invalid syntax (<ipython-input-35-13145b7dbde5>, line 1)

In [33]:
ds['InvoiceNo'].dtypes

dtype('O')

In [34]:
for row in ds.iterrows():
    print (row['InvoiceNo'].startswith('c'))

TypeError: tuple indices must be integers or slices, not str

In [36]:
ds['StockCode'].value_counts()

85123A    2077
22423     1905
85099B    1662
84879     1418
47566     1416
20725     1359
22720     1232
POST      1196
20727     1126
22197     1118
23203     1115
22383     1103
21212     1080
23209     1040
23298     1036
22086     1029
22382     1021
20728     1012
22457      997
22469      996
22384      977
22960      974
21034      955
22727      932
23206      907
82482      905
22386      903
22993      902
22138      894
22666      890
          ... 
90033        1
90039A       1
84016        1
90028        1
62096B       1
90074        1
90078        1
82615        1
90135        1
84856S       1
90179C       1
90152B       1
35607A       1
79323W       1
90214U       1
84705C       1
90176C       1
90138        1
90131        1
90039C       1
17001        1
90104        1
90190A       1
84743C       1
90102        1
90024F       1
90100        1
44242A       1
44265        1
90163B       1
Name: StockCode, Length: 3684, dtype: int64